<a href="https://colab.research.google.com/github/manki8908/firstcompany/blob/master/15_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import folium
import matplotlib.pyplot as plt
import matplotlib as mpl

In [3]:
df = pd.read_csv('/content/자치구별 공원 현황.csv')
df.head()

,지역,공원면적,공원수,인구,면적,면적비율,인당면적
0,강남구,3626398,7,544055,39500000.0,9.180754,6.665499
1,강동구,1526390,7,463998,24590000.0,6.207361,3.289648
2,강북구,3714902,4,311569,23600000.0,15.741110,11.923208
3,강서구,2194154,9,585901,41440000.0,5.294773,3.744923
4,관악구,10431453,2,509803,29570000.0,35.277149,20.461733


# 1. 자치구별 면적대비/인구대비 공원 면적

## 맵 로드

In [ ]:
# 동별 geo 데이터
geo_path = '/content/seoul-dong.geojson'
try: 
    df_geo = json.load(open(geo_path, encoding='utf-8'))
except:
    df_geo = json.load(open(geo_path, econding='utf-8-sig'))   # return dict

df_geo.keys()
#df_geo
df_geo['features'][0]

In [ ]:
# 구별 geo 데이터
geo_gu_path = '/content/seoul-gu-simple.json'
try: 
    df_geo_gu = json.load(open(geo_gu_path, encoding='utf-8'))
except:
    df_geo_gu = json.load(open(geo_gu_path, econding='utf-8-sig'))   # return dict

df_geo_gu.keys()
df_geo_gu['features'][0]

In [26]:
# 데이터 처리
new_df1 = df[['지역','면적비율']]
new_df1.head()

# 데이터 처리
new_df2 = df[['지역','인당면적']]
new_df2.head()

# 위경도 데이터
ll = pd.read_csv('서울공원요약.csv', encoding='cp949')
ll

,공원명,지역,경도,위도,면적,스케일,크기
0,남산도시자연공원,중구,126.990377,37.550140,896887,중형,7
1,길동생태공원,강동구,127.154779,37.540394,80683,소형,3
2,서울대공원,과천시,127.019846,37.426449,913690,중형,7
3,서울숲,성동구,127.041798,37.543072,480994,중형,7
4,월드컵공원,마포구,126.878907,37.571805,84085,소형,3
...,...,...,...,...,...,...,...
125,숭인근린공원,종로구,127.017827,37.576442,45615,소형,3
126,서일대뒷산공원,중랑구,127.098782,37.587716,63101,소형,3
127,용마도시자연공원(사가정공원),중랑구,127.095997,37.579762,109635,중형,7
128,문화비축기지,마포구,126.893246,37.571718,1400,소형,3


## 단계구분도

### 단계구분도 - 자치구별 면적비율 

In [ ]:


# 맵
#map = folium.Map(location=[37.581,126.986], zoom_start=11, tiles='Stamen Toner')
map = folium.Map(location=[37.581,126.986], zoom_start=11)
#folium.TileLayer('cartodbpositron').add_to(map)

# 단계구분도
folium.Choropleth(geo_data = df_geo_gu,                     # 지도 데이터 파일
                  data = new_df1,                         # 시각화 데이터 파일
                  columns = ('지역', '면적비율'),              # (지도 데이터와 맵핑할 값, 시각화 하고자하는 변수)
                  key_on='feature.properties.name',        # 데이터파일과 매칭할 값
                  fill_color = 'BuPu', 
                  legend_name = '면적비율',
                  ).add_to(map)
title = '<h3 align="center" style="font-size:20px"> 면적비율</h3>'
map.get_root().html.add_child(folium.Element(title))
map

### 단계구분도 - 자치구별 인구비율 

In [ ]:
# 맵
#map = folium.Map(location=[37.581,126.986], zoom_start=11, tiles='Stamen Toner')
map = folium.Map(location=[37.581,126.986], zoom_start=11)
#folium.TileLayer('cartodbpositron').add_to(map)

# 단계구분도
folium.Choropleth(geo_data = df_geo_gu,                     # 지도 데이터 파일
                  data = new_df2,                         # 시각화 데이터 파일
                  columns = ('지역', '인당면적'),              # (지도 데이터와 맵핑할 값, 시각화 하고자하는 변수)
                  key_on='feature.properties.name',        # 데이터파일과 매칭할 값
                  fill_color = 'BuPu', 
                  legend_name = '인당면적',
                  ).add_to(map)
title = '<h3 align="center" style="font-size:20px"> 인당면적</h3>'
map.get_root().html.add_child(folium.Element(title))
map

### 공원위치 추가표시

In [32]:
# 공원크기 
for i in ll.index:
  folium.Circle(radius=int(ll.크기[i]*100),
                location=[ll.위도[i], ll.경도[i]],
                popup=folium.Popup(ll.공원명[i], max_width=200),
                tooltip = f"{ll.공원명[i]}({ll.면적[i]: ,d})㎡",
                color='red',
                fill = True,
                fill_color='#3186cc').add_to(map)
map